In [ ]:
# Note Cell to clarify changes done to notebook to cope with #Episode or #EP class_mode = 'episode) related changes 
# Or #Transformative or #TR for differences due to dataset being Atari
# and trying to teach the network time with a new dimention on input data

# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Windows import
os.chdir('..' + sep_local+'..' + sep_local + '..' + sep_local +'..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
/
/home/azeghost/git/azeghost/Generative_Models


In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.1


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#images_dir = 'C:\\projects\\pokemon\DS06\\'
validation_percentage = 25
valid_format = 'png'

### inputs_shape changed to (EPIS_LEN, ) + image_size due to class_mode='episode' when data is loaded

In [5]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= (EPIS_LEN, ) + image_size
latents_dim = 30
intermediate_dim = 30

In [8]:
from training.generators.from_images.file_image_generator import create_image_lists, get_generators

In [9]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 0 
)

In [10]:
#we created new  class_mode  episode_flat witch returns 
#da.from_array(np.reshape(batch_x, (-1,)+self.image_shape )), da.from_array(np.reshape(batch_gt, (-1,)+self.image_shape))

### Class_mode is episode because time dimention "TimeDistributed" of the model

In [11]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode',
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

  INFO     | Found 1172401 training files

  INFO     | Found 381757 validation files



In [12]:
import tensorflow as tf
import numpy as np

### #Transformative changed output shapes of Dataset Generators 
from (batch_size, EPIS_LEN, ) + image_size)
to (batch_size* EPIS_LEN, ) + image_size) 

In [13]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
                  )
)


In [14]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [15]:
_instance_scale = 1.0

In [16]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod((EPIS_LEN,) + inputs_shape)
inputs_shape    

(10, 160, 210, 3)

# Model's Layers definition

In [17]:
tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))

In [18]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

### #Transformative Dense layer units=latents_dim*EPIS_LEN meaning 30 due to uniformity between all networks  target_shape=(EPIS_LEN,) + c

In [19]:
units=30
c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2)
menc_lays = [
    tdConv(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), # units*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(units=latents_dim)
]

venc_lays  = [
    tdConv(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), # units*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(units=latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product((EPIS_LEN,) + c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(EPIS_LEN,) + c),
    tdDeConv(filters=units//5, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
    tdDeConv(filters=units, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tdDeConv(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition


In [20]:
import os
model_name = 'Atari_VAE_TimeDist_Conv'
#experiments_dir='..'+sep_local+'..'+sep_local+'..'+sep_local+'experiments'+sep_local + model_name

experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [21]:
from training.autoencoding_basic.transformative.VAE import VAE as AE

### #Transformative 'outputs_shape':latents_dim*EPIS_LEN,

In [22]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    }

    ,
    
        {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [23]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("experiments_dir",absolute)
print("Current working dir",os.getcwd())

Restore_dir /home/azeghost/git/azeghost/Generative_Models/experiments/Atari_VAE_TimeDist_Conv/var_save_dir
experiments_dir /home/azeghost/git/azeghost/Generative_Models/experiments/Atari_VAE_TimeDist_Conv
Current working dir /home/azeghost/git/azeghost/Generative_Models


 ## Due to Time Dimension do not multiply batch_size with EPIS_LEN

In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size*EPIS_LEN,
    episode_len= 1, 
    # episode_len= EPIS_LEN,    
    variables_params=variables_params, 
    filepath=None
    )

  WARNING  | None
  WARNING  | None
Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 10, 160, 210, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 79, 104, 30)   840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 39, 34, 6)     1626      
_________________________________________________________________
flatten (Flatten)            (None, 79560)             0         
_________________________________________________________________
dense (Dense)                (None, 30)                2386830   
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
_________________________________________________________________
dropout (Dropout

## Problem with 
self.ae_metrics = create_metrics([self.batch_size] + self.get_outputs_shape()[-3:])
Becuse if we increase the batch_size to become batch_Size * EPIS_len
The following problem will occur ::

imageB Tensor("metrics/x_logits_psnr/Sigmoid:0", shape=(100, 10, 160, 210, 3), dtype=float32)
inputs_flat_shape [100, 160, 210, 3]

In [27]:
# ae.compile(metrics=None)
ae.compile()

Model: "Atari_VAE_TimeDist_Conv"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 10, 160, 210 0                                            
__________________________________________________________________________________________________
inference_logvariance (Function (None, 30)           2389416     inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 30)]         0           inference_logvariance[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 30)]         0           tf_op_layer_Mul[0][0]            
____________________________________________________________________________

In [ ]:
ae.get_inputs_shape(),ae.get_outputs_shape()

In [ ]:
print()

In [ ]:
list(ae.get_variables()['inference_mean'].inputs[0].shape)

In [ ]:
 [ae.batch_size] + ae.get_outputs_shape()

In [ ]:
ae.get_variables()['generative'].outputs[0].shape

# Callbacks

In [ ]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

### #Transformative latents_shape should be  latents_dim* EPIS_LEN

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=10,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=10
)

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)